# CMOR example with prototype CMIP7 tables/CVs

This code is a demonstration of the use of CMOR with prototype CMIP7 CVs and tables. This will require a conda environment in which CMOR 3.10.0 has been installed

Please note that file and directory names required for CMIP7 have not been confirmed yet, this is just a limited demonstration.

PJD  9 Jun 2025 - updated to redefine `license` -> `license_id` in the DATASET_INFO

In [1]:
%time
import cmor
import json
import numpy
import os

CPU times: user 5 μs, sys: 2 μs, total: 7 μs
Wall time: 14.1 μs


The following is adapted from the code at https://github.com/PCMDI/cmor/blob/main/Test/test_cmor_CMIP7.py

# Get CWD

In [2]:
print(os.getcwd())

/Users/durack1/sync/git/cmip7-development-mip-tables


# Construct the CMOR input json file

In [3]:
%time
DATASET_INFO = {
    "_AXIS_ENTRY_FILE": "development-tables/CMIP7_coordinate.json",
    "_FORMULA_VAR_FILE": "development-tables/CMIP7_formula_terms.json",
    "_cmip7_option": 1,
    "_controlled_vocabulary_file": "development-tables/CMIP7_CV.json",
    "activity_id": "CMIP",
    "branch_method": "standard",
    "branch_time_in_child": 30.0,
    "branch_time_in_parent": 10800.0,
    "calendar": "360_day",
    "cv_version": "7.0.0.0",
    "experiment": "1 percent per year increase in CO2",
    "experiment_id": "1pctCO2",
    "forcing_index": "3",
    "grid": "N96",
    "grid_label": "gn",
    "initialization_index": "1",
    "institution_id": "PCMDI",
    "license_id": "CC BY 4.0",
    "nominal_resolution": "250 km",
    "outpath": ".",
    "parent_mip_era": "CMIP7",
    "parent_time_units": "days since 1850-01-01",
    "parent_activity_id": "CMIP",
    "parent_source_id": "PCMDI-test-1-0",
    "parent_experiment_id": "piControl",
    "parent_variant_label": "r1i1p1f3",
    "physics_index": "1",
    "realization_index": "9",
    "source_id": "PCMDI-test-1-0",
    "source_type": "AOGCM CHEM BGC",
    "tracking_prefix": "hdl:21.14100",
    "host_collection": "CMIP7",
    "frequency": "day",
    "region": "glb",
    "archive_id": "WCRP",
    "mip_era": "CMIP7",
    "output_path_template": "<mip_era><activity_id><source_id><experiment_id><member_id><realm><frequency><variable_id><branding_suffix><grid_label><version>",
    "output_file_template": "<variable_id><branding_suffix><frequency><region><grid_label><source_id><experiment_id><variant_label>",
}

with open('input.json', 'w') as fh:
    json.dump(DATASET_INFO, fh, indent=2)

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 3.1 μs


In [4]:
%time
cmor.setup(inpath="development-tables", netcdf_file_action=cmor.CMOR_REPLACE)

cmor.dataset_json('input.json')

tos = numpy.array([27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27
                    ])
tos.shape = (2, 3, 4)
lat = numpy.array([10, 20, 30])
lat_bnds = numpy.array([5, 15, 25, 35])
lon = numpy.array([0, 90, 180, 270])
lon_bnds = numpy.array([-45, 45,
                        135,
                        225,
                        315
                        ])
time = numpy.array([15.5, 16.5])
time_bnds = numpy.array([15, 16, 17])

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 1.91 μs


In [5]:
%time
cmor.load_table("CMIP7_ocean.json")

CPU times: user 1e+03 ns, sys: 1 μs, total: 2 μs
Wall time: 3.1 μs



C Traceback:
In function: cmor_load_table_internal
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: unknown section: version_metadata, for table: ocean
!
!!!!!!!!!!!!!!!!!!!!!!!!!


C Traceback:
In function: cmor_load_table_internal
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: attribute for unknown section: version_metadata,{ "version_tag": "v6.5.0.0", "checksum": "md5: c4c59dea7b58b910a6dc5345e31638f6", "Auxillary_files\/MIP_formula_terms_modified": "Tue Nov 28 21:14:07 2023 +0000", "Auxillary_files\/MIP_formula_terms_note": "Updating versioning and model_components scripts to process files individually.", "commit": "50ee06a3b58fb2b064485d454203766327f4d770", "previous_commit": "", "author": "Daniel Ellis", "CV_collection_version": "v6.5.1.1", "specs_doc": "v6.5.0" } (table: ocean)
!
!!!!!!!!!!!!!!!!!!!!!!!!!



0

In [6]:
%time
cmorlat = cmor.axis("latitude",
                    coord_vals=lat,
                    cell_bounds=lat_bnds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=lon,
                    cell_bounds=lon_bnds,
                    units="degrees_east")
cmortime = cmor.axis("time",
                    coord_vals=time,
                    cell_bounds=time_bnds,
                    units="days since 2018")
axes = [cmortime, cmorlat, cmorlon]

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 μs


In [7]:
cmortos = cmor.variable("tos_tavg-u-hxy-sea", "degC", axes)

In [8]:
cmor.write(cmortos, tos)
filename = cmor.close(cmortos, file_name=True)


C Traceback:
In function: cmor_close_variable
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: while closing variable 0 (tos, table ocean)
! we noticed you wrote 0 time steps for the variable,
! but its time axis 0 (time) has 2 time steps
!
!!!!!!!!!!!!!!!!!!!!!!!!!



# Locate generated file

In [9]:
!find CMIP7 -type f -name *.nc

CMIP7/CMIP/PCMDI-test-1-0/1pctCO2/r9i1p1f3/ocean/day/tos/tavg-u-hxy-sea/gn/v20250609/tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_1pctCO2_r9i1p1f3_20180116-20180117.nc


# Dump header of generated file

In [10]:
print(os.getcwd())
!ncdump -h CMIP7/CMIP/PCMDI-test-1-0/1pctCO2/r9i1p1f3/ocean/day/tos/tavg-u-hxy-sea/gn/*/tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_1pctCO2_r9i1p1f3_20180116-20180117.nc

/Users/durack1/sync/git/cmip7-development-mip-tables
netcdf tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_1pctCO2_r9i1p1f3_20180116-20180117 {
dimensions:
	time = UNLIMITED ; // (2 currently)
	lat = 3 ;
	lon = 4 ;
	bnds = 2 ;
variables:
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 2018" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
		time:long_name = "Time Intervals" ;
		time:standard_name = "time" ;
	double time_bnds(time, bnds) ;
	double lat(lat) ;
		lat:bounds = "lat_bnds" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "Latitude" ;
		lat:standard_name = "latitude" ;
	double lat_bnds(lat, bnds) ;
	double lon(lon) ;
		lon:bounds = "lon_bnds" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "Longitude" ;
		lon:standard_name = "longitude" ;
	double lon_bnds(lon, bnds) ;
	float tos(time, lat, lon) ;
		tos:standard_name = "sea_surface_temperature" ;
		tos:long_name = "Sea Surface Temperature" ;
		tos:brand_

In [11]:
# clean up
#! rm -rf CMIP7
#! rm -rf input.json